In [2]:
import cv2
import os
import numpy as np

In [2]:
def read_video_volume(volume_folder, num_frames=10):
    frames = []
    for i in range(num_frames):
        frame_path = os.path.join(volume_folder, f'frame_{i:04d}.png')
        frame = cv2.imread(frame_path)
        if frame is None:
            raise ValueError(f"Error: Could not read frame: {frame_path}")
        frames.append(frame)
    return frames

In [3]:
def compute_optical_flow_farneback(frames):
    flows = []
    prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)
    
    for i in range(1, len(frames)):
        curr_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 
                                            0.5, 3, 15, 3, 5, 1.2, 0)
        flows.append(flow)
        prev_gray = curr_gray
    
    return flows

In [4]:
def save_optical_flow(flows, frames, output_folder):
    if not os.path.exists(output_folder):
        print(f"Creating output directory: {output_folder}")
        os.makedirs(output_folder)
    else:
        print(f"Output directory already exists: {output_folder}")
    
    for i, flow in enumerate(flows):
        # Compute magnitude and angle of the flow
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        
        # Create an HSV image
        hsv = np.zeros_like(frames[0])
        hsv[..., 1] = 255
        
        # Set hue and value
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        
        # Convert HSV to BGR
        bgr_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
        # Save the flow image
        flow_filename = os.path.join(output_folder, f'flow_{i:04d}.png')
        cv2.imwrite(flow_filename, bgr_flow)
        print(f"Saved optical flow image: {flow_filename}")

In [5]:
def process_video_volumes(input_folder, output_folder, num_frames=10):
    if not os.path.exists(input_folder):
        print(f"Input folder does not exist: {input_folder}")
        return
    
    for subfolder in os.listdir(input_folder):
        subfolder_path = os.path.join(input_folder, subfolder)
        if os.path.isdir(subfolder_path):
            for volume_folder in os.listdir(subfolder_path):
                volume_path = os.path.join(subfolder_path, volume_folder)
                if os.path.isdir(volume_path):
                    print(f"Processing volume: {volume_path}")
                    try:
                        frames = read_video_volume(volume_path, num_frames)
                        print(f"Read {len(frames)} frames from {volume_path}")
                        flows = compute_optical_flow_farneback(frames)
                        print(f"Computed optical flows for {volume_path}")
                        output_volume_folder = os.path.join(output_folder, subfolder, volume_folder)
                        save_optical_flow(flows, frames, output_volume_folder)
                    except Exception as e:
                        print(f"Error processing volume {volume_path}: {e}")

In [6]:
# Input and output paths
input_folder = '/kaggle/input/d/pranshulatri/video-vol-for-optical-flow/video_volumes'
output_folder = '/kaggle/working/optical_flows'

In [7]:
process_video_volumes(input_folder, output_folder)

In [3]:
import cv2
import os
import numpy as np

def read_video_volume(volume_folder, num_frames=10):
    frames = []
    for i in range(num_frames):
        frame_path = os.path.join(volume_folder, f'frame_{i:04d}.png')
        frame = cv2.imread(frame_path)
        if frame is None:
            print(f"Error: Could not read frame: {frame_path}")
            return None
        frames.append(frame)
    return frames

def compute_optical_flow_farneback(frames):
    flows = []
    prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)
    
    for i in range(1, len(frames)):
        curr_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 
                                            0.5, 3, 15, 3, 5, 1.2, 0)
        flows.append(flow)
        prev_gray = curr_gray
    
    return flows

def save_optical_flow(flows, frames, output_folder):
    if not os.path.exists(output_folder):
        print(f"Creating output directory: {output_folder}")
        os.makedirs(output_folder)
    else:
        print(f"Output directory already exists: {output_folder}")
    
    for i, flow in enumerate(flows):
        # Compute magnitude and angle of the flow
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        
        # Create an HSV image
        hsv = np.zeros_like(frames[0])
        hsv[..., 1] = 255
        
        # Set hue and value
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        
        # Convert HSV to BGR
        bgr_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
        # Save the flow image
        flow_filename = os.path.join(output_folder, f'flow_{i:04d}.png')
        cv2.imwrite(flow_filename, bgr_flow)
        print(f"Saved optical flow image: {flow_filename}")

def process_video_volumes(input_folder, output_folder, num_frames=10):
    if not os.path.exists(input_folder):
        print(f"Input folder does not exist: {input_folder}")
        return
    
    for subfolder in os.listdir(input_folder):
        subfolder_path = os.path.join(input_folder, subfolder)
        if os.path.isdir(subfolder_path):
            for volume_folder in os.listdir(subfolder_path):
                volume_path = os.path.join(subfolder_path, volume_folder)
                if os.path.isdir(volume_path):
                    print(f"Processing volume: {volume_path}")
                    frames = read_video_volume(volume_path, num_frames)
                    if frames is None:
                        print(f"Skipping volume {volume_path} due to read error")
                        continue
                    print(f"Read {len(frames)} frames from {volume_path}")
                    flows = compute_optical_flow_farneback(frames)
                    print(f"Computed optical flows for {volume_path}")
                    output_volume_folder = os.path.join(output_folder, subfolder, volume_folder)
                    save_optical_flow(flows, frames, output_volume_folder)

# Input and output paths
input_folder = '/kaggle/input/d/pranshulatri/video-vol-for-optical-flow/video_volumes'
output_folder = '/kaggle/working/optical_flows'

process_video_volumes(input_folder, output_folder)

In [4]:
import os

def print_directory_structure(root_dir):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        level = dirpath.replace(root_dir, '').count(os.sep)
        indent = ' ' * 4 * level
        print(f"{indent}{os.path.basename(dirpath)}/")
        subindent = ' ' * 4 * (level + 1)
        for f in filenames:
            print(f"{subindent}{f}")

input_folder = '/kaggle/input/d/pranshulatri/video-vol-for-optical-flow/video_volumes'
print_directory_structure(input_folder)

video_volumes/
    volume_35_2/
        frame_0002.png
        frame_0000.png
        frame_0003.png
        frame_0001.png
        frame_0005.png
        frame_0006.png
        frame_0008.png
        frame_0007.png
        frame_0009.png
        frame_0004.png
    volume_84_1/
        frame_0002.png
        frame_0000.png
        frame_0003.png
        frame_0001.png
        frame_0005.png
        frame_0006.png
        frame_0008.png
        frame_0007.png
        frame_0009.png
        frame_0004.png
    volume_110_9/
        frame_0002.png
        frame_0000.png
        frame_0003.png
        frame_0001.png
        frame_0005.png
        frame_0006.png
        frame_0008.png
        frame_0007.png
        frame_0009.png
        frame_0004.png
    volume_68_2/
        frame_0002.png
        frame_0000.png
        frame_0003.png
        frame_0001.png
        frame_0005.png
        frame_0006.png
        frame_0008.png
        frame_0007.png
        frame_0009.png
        frame_0004.

# **RUN FROM HERE**

**This is for sorting the dataset**

In [2]:
import os

def organize_dataset(root_folder):
    volumes = sorted([d for d in os.listdir(root_folder) if os.path.isdir(os.path.join(root_folder, d))])
    for volume in volumes:
        volume_path = os.path.join(root_folder, volume)
        images = sorted([f for f in os.listdir(volume_path) if os.path.isfile(os.path.join(volume_path, f))])
        for i, image in enumerate(images):
            old_image_path = os.path.join(volume_path, image)
            new_image_path = os.path.join(volume_path, f'frame_{i:04d}.png')
            os.rename(old_image_path, new_image_path)
        print(f"Organized {volume_path}")

# Path to the dataset root folder
dataset_root_folder = '/kaggle/input/d/pranshulatri/video-vol-for-optical-flow/video_volumes'
organize_dataset(dataset_root_folder)

OSError: [Errno 30] Read-only file system: '/kaggle/input/d/pranshulatri/video-vol-for-optical-flow/video_volumes/volume_0_0/frame_0000.png' -> '/kaggle/input/d/pranshulatri/video-vol-for-optical-flow/video_volumes/volume_0_0/frame_0000.png'

**Printing the directory structure**

In [3]:
import os

def print_directory_structure(root_dir):
    for dirpath, dirnames, filenames in os.walk(root_dir):
        level = dirpath.replace(root_dir, '').count(os.sep)
        indent = ' ' * 4 * level
        print(f"{indent}{os.path.basename(dirpath)}/")
        subindent = ' ' * 4 * (level + 1)
        for f in filenames:
            print(f"{subindent}{f}")

input_folder = '/kaggle/input/d/pranshulatri/video-vol-for-optical-flow/video_volumes'
print_directory_structure(input_folder)

video_volumes/
    volume_35_2/
        frame_0002.png
        frame_0000.png
        frame_0003.png
        frame_0001.png
        frame_0005.png
        frame_0006.png
        frame_0008.png
        frame_0007.png
        frame_0009.png
        frame_0004.png
    volume_84_1/
        frame_0002.png
        frame_0000.png
        frame_0003.png
        frame_0001.png
        frame_0005.png
        frame_0006.png
        frame_0008.png
        frame_0007.png
        frame_0009.png
        frame_0004.png
    volume_110_9/
        frame_0002.png
        frame_0000.png
        frame_0003.png
        frame_0001.png
        frame_0005.png
        frame_0006.png
        frame_0008.png
        frame_0007.png
        frame_0009.png
        frame_0004.png
    volume_68_2/
        frame_0002.png
        frame_0000.png
        frame_0003.png
        frame_0001.png
        frame_0005.png
        frame_0006.png
        frame_0008.png
        frame_0007.png
        frame_0009.png
        frame_0004.

**Main Code Farneback implementaiton**

In [22]:
import cv2
import os
import numpy as np

def read_video_volume(volume_folder, num_frames=10):
    frames = []
    for i in range(num_frames):
        frame_path = os.path.join(volume_folder, f'frame_{i:04d}.png')
        frame = cv2.imread(frame_path)
        if frame is None:
            print(f"Error: Could not read frame: {frame_path}")
            return None
        frames.append(frame)
    return frames

def compute_optical_flow_farneback(frames):
    flows = []
    prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)
    
    for i in range(1, len(frames)):
        curr_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        flow = cv2.calcOpticalFlowFarneback(prev_gray, curr_gray, None, 
                                            0.5, 3, 15, 3, 5, 1.2, 0)
        flows.append(flow)
        prev_gray = curr_gray
    
    return flows

def save_optical_flow(flows, frames, output_folder):
    if not os.path.exists(output_folder):
        print(f"Creating output directory: {output_folder}")
        os.makedirs(output_folder)
    else:
        print(f"Output directory already exists: {output_folder}")
    
    for i, flow in enumerate(flows):
        # Compute magnitude and angle of the flow
        mag, ang = cv2.cartToPolar(flow[..., 0], flow[..., 1])
        
        # Create an HSV image
        hsv = np.zeros_like(frames[0])
        hsv[..., 1] = 255
        
        # Set hue and value
        hsv[..., 0] = ang * 180 / np.pi / 2
        hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX)
        
        # Convert HSV to BGR
        bgr_flow = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        
        # Save the flow image
        flow_filename = os.path.join(output_folder, f'flow_{i:04d}.png')
        cv2.imwrite(flow_filename, bgr_flow)
        print(f"Saved optical flow image: {flow_filename}")

def process_video_volumes(input_folder, output_folder, num_frames=10):
    if not os.path.exists(input_folder):
        print(f"Input folder does not exist: {input_folder}")
        return
    
    for volume_folder in sorted(os.listdir(input_folder)):
        volume_path = os.path.join(input_folder, volume_folder)
        if os.path.isdir(volume_path):
            print(f"Processing volume: {volume_path}")
            frames = read_video_volume(volume_path, num_frames)
            if frames is None:
                print(f"Skipping volume {volume_path} due to read error")
                continue
            print(f"Read {len(frames)} frames from {volume_path}")
            flows = compute_optical_flow_farneback(frames)
            print(f"Computed optical flows for {volume_path}")
            output_volume_folder = os.path.join(output_folder, volume_folder)
            save_optical_flow(flows, frames, output_volume_folder)

# Input and output paths
input_folder = 'C:\\Users\hci\\Desktop\\Data1\\V21\\video_volumes'
output_folder = 'C:\\Users\hci\\Desktop\\Data1\\V21\\optical_flows'

process_video_volumes(input_folder, output_folder)

Processing volume: C:\Users\hci\Desktop\Data1\V21\video_volumes\volume_0_0
Read 10 frames from C:\Users\hci\Desktop\Data1\V21\video_volumes\volume_0_0
Computed optical flows for C:\Users\hci\Desktop\Data1\V21\video_volumes\volume_0_0
Creating output directory: C:\Users\hci\Desktop\Data1\V21\optical_flows\volume_0_0
Saved optical flow image: C:\Users\hci\Desktop\Data1\V21\optical_flows\volume_0_0\flow_0000.png
Saved optical flow image: C:\Users\hci\Desktop\Data1\V21\optical_flows\volume_0_0\flow_0001.png
Saved optical flow image: C:\Users\hci\Desktop\Data1\V21\optical_flows\volume_0_0\flow_0002.png
Saved optical flow image: C:\Users\hci\Desktop\Data1\V21\optical_flows\volume_0_0\flow_0003.png
Saved optical flow image: C:\Users\hci\Desktop\Data1\V21\optical_flows\volume_0_0\flow_0004.png
Saved optical flow image: C:\Users\hci\Desktop\Data1\V21\optical_flows\volume_0_0\flow_0005.png
Saved optical flow image: C:\Users\hci\Desktop\Data1\V21\optical_flows\volume_0_0\flow_0006.png
Saved optic

**Main code Lucas-Kanade: Sparse Optical Flow approach**

In [1]:
# import cv2
# import os
# import numpy as np

# def read_video_volume(volume_folder, num_frames=10):
#     frames = []
#     for i in range(num_frames):
#         frame_path = os.path.join(volume_folder, f'frame_{i:04d}.png')
#         frame = cv2.imread(frame_path)
#         if frame is None:
#             print(f"Error: Could not read frame: {frame_path}")
#             return None
#         frames.append(frame)
#     return frames

# def compute_optical_flow_lucas_kanade(frames):
#     flows = []
#     prev_gray = cv2.cvtColor(frames[0], cv2.COLOR_BGR2GRAY)
    
#     # Parameters for ShiTomasi corner detection
#     feature_params = dict(maxCorners=100, qualityLevel=0.3, minDistance=7, blockSize=7)
#     # Parameters for Lucas Kanade optical flow
#     lk_params = dict(winSize=(15, 15), maxLevel=2,
#                      criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
    
#     # Create some random colors
#     color = np.random.randint(0, 255, (100, 3))

#     # Find initial points to track
#     p0 = cv2.goodFeaturesToTrack(prev_gray, mask=None, **feature_params)
#     if p0 is None:
#         print("Error: No corners found in the first frame.")
#         return []
    
#     # Create a mask image for drawing purposes
#     mask = np.zeros_like(frames[0])
    
#     for i in range(1, len(frames)):
#         curr_gray = cv2.cvtColor(frames[i], cv2.COLOR_BGR2GRAY)
        
#         # Calculate optical flow
#         p1, st, err = cv2.calcOpticalFlowPyrLK(prev_gray, curr_gray, p0, None, **lk_params)
        
#         if p1 is None or st is None:
#             print(f"Error: Optical flow calculation failed for frame {i}.")
#             continue
        
#         # Select good points
#         good_new = p1[st == 1]
#         good_old = p0[st == 1]
        
#         # Draw the tracks
#         for j, (new, old) in enumerate(zip(good_new, good_old)):
#             a, b = new.ravel()
#             c, d = old.ravel()
#             a, b, c, d = map(int, (a, b, c, d))
#             mask = cv2.line(mask, (a, b), (c, d), color[j].tolist(), 2)
#             frames[i] = cv2.circle(frames[i], (a, b), 5, color[j].tolist(), -1)
        
#         img = cv2.add(frames[i], mask)
#         flows.append(img)
#         prev_gray = curr_gray.copy()
#         p0 = good_new.reshape(-1, 1, 2)
    
#     return flows

# def save_optical_flow(flows, output_folder):
#     if not os.path.exists(output_folder):
#         print(f"Creating output directory: {output_folder}")
#         os.makedirs(output_folder)
#     else:
#         print(f"Output directory already exists: {output_folder}")
    
#     for i, flow in enumerate(flows):
#         # Save the flow image
#         flow_filename = os.path.join(output_folder, f'flow_{i:04d}.png')
#         cv2.imwrite(flow_filename, flow)
#         print(f"Saved optical flow image: {flow_filename}")

# def process_video_volumes(input_folder, output_folder, num_frames=10):
#     if not os.path.exists(input_folder):
#         print(f"Input folder does not exist: {input_folder}")
#         return
    
#     for volume_folder in sorted(os.listdir(input_folder)):
#         volume_path = os.path.join(input_folder, volume_folder)
#         if os.path.isdir(volume_path):
#             print(f"Processing volume: {volume_path}")
            
#             frames = read_video_volume(volume_path, num_frames)
#             if frames is None:
#                 print(f"Skipping volume {volume_path} due to read error")
#                 continue
#             print(f"Read {len(frames)} frames from {volume_path}")
#             flows = compute_optical_flow_lucas_kanade(frames)
#             if not flows:
#                 print(f"No optical flows computed for {volume_path}")
#                 continue
#             print(f"Computed optical flows for {volume_path}")
#             output_volume_folder = os.path.join(output_folder, volume_folder)
#             save_optical_flow(flows, output_volume_folder)

# # Input and output paths
# input_folder = '/kaggle/input/d/pranshulatri/video-vol-for-optical-flow/video_volumes'
# output_folder = '/kaggle/working/optical_flows'

# process_video_volumes(input_folder, output_folder)

Processing volume: /kaggle/input/d/pranshulatri/video-vol-for-optical-flow/video_volumes/volume_0_0
Read 10 frames from /kaggle/input/d/pranshulatri/video-vol-for-optical-flow/video_volumes/volume_0_0
Computed optical flows for /kaggle/input/d/pranshulatri/video-vol-for-optical-flow/video_volumes/volume_0_0
Creating output directory: /kaggle/working/optical_flows/volume_0_0
Saved optical flow image: /kaggle/working/optical_flows/volume_0_0/flow_0000.png
Saved optical flow image: /kaggle/working/optical_flows/volume_0_0/flow_0001.png
Saved optical flow image: /kaggle/working/optical_flows/volume_0_0/flow_0002.png
Saved optical flow image: /kaggle/working/optical_flows/volume_0_0/flow_0003.png
Saved optical flow image: /kaggle/working/optical_flows/volume_0_0/flow_0004.png
Saved optical flow image: /kaggle/working/optical_flows/volume_0_0/flow_0005.png
Saved optical flow image: /kaggle/working/optical_flows/volume_0_0/flow_0006.png
Saved optical flow image: /kaggle/working/optical_flows/

In [2]:
import shutil

# Specify the path to your folder
folder_path = '/kaggle/working/optical_flows'
zip_file_path = '/kaggle/working/optical_flows.zip'

# Create a zip archive of the folder
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

'/kaggle/working/optical_flows.zip'

**Combining back the optical video volumes**

In [4]:
import cv2
import os
import numpy as np

def combine_frames(frames_list, frame_shape):
    combined_frame = np.zeros((frame_shape[0] * 2, frame_shape[1] * 5, frame_shape[2]), dtype=np.uint8)
    
    for i, frame in enumerate(frames_list):
        row_start = (i // 5) * frame_shape[0]
        col_start = (i % 5) * frame_shape[1]
        combined_frame[row_start:row_start + frame_shape[0], col_start:col_start + frame_shape[1]] = frame
    
    return combined_frame

def reassemble_video_volumes(input_folder, output_folder, num_volumes=10, num_frames=9):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for main_volume in range(122):
        output_volume_folder = os.path.join(output_folder, f'volume_{main_volume}')
        if not os.path.exists(output_volume_folder):
            os.makedirs(output_volume_folder)
        
        for frame_num in range(num_frames):
            frames_list = []
            for sub_volume in range(num_volumes):
                frame_path = os.path.join(input_folder, f'volume_{main_volume}_{sub_volume}', f'flow_{frame_num:04d}.png')
                print(f"Checking frame path: {frame_path}")
                if not os.path.exists(frame_path):
                    print(f"Error: File does not exist: {frame_path}")
                    continue
                frame = cv2.imread(frame_path)
                if frame is None:
                    print(f"Error: Could not read frame: {frame_path}")
                    continue
                frames_list.append(frame)
            
            if len(frames_list) == num_volumes:
                combined_frame = combine_frames(frames_list, frames_list[0].shape)
                output_frame_path = os.path.join(output_volume_folder, f'flow_{frame_num:04d}.png')
                cv2.imwrite(output_frame_path, combined_frame)
                print(f"Saved combined frame: {output_frame_path}")
            else:
                print(f"Error: Incomplete frames for volume {main_volume}, frame {frame_num}")

# Input and output paths
input_folder = '/kaggle/input/optical-flow-recombine/optical_flows'
output_folder = '/kaggle/working/reassembled_video_volumes'

reassemble_video_volumes(input_folder, output_folder)

Checking frame path: /kaggle/input/optical-flow-recombine/optical_flows/volume_0_0/flow_0000.png
Checking frame path: /kaggle/input/optical-flow-recombine/optical_flows/volume_0_1/flow_0000.png
Checking frame path: /kaggle/input/optical-flow-recombine/optical_flows/volume_0_2/flow_0000.png
Checking frame path: /kaggle/input/optical-flow-recombine/optical_flows/volume_0_3/flow_0000.png
Checking frame path: /kaggle/input/optical-flow-recombine/optical_flows/volume_0_4/flow_0000.png
Checking frame path: /kaggle/input/optical-flow-recombine/optical_flows/volume_0_5/flow_0000.png
Checking frame path: /kaggle/input/optical-flow-recombine/optical_flows/volume_0_6/flow_0000.png
Checking frame path: /kaggle/input/optical-flow-recombine/optical_flows/volume_0_7/flow_0000.png
Checking frame path: /kaggle/input/optical-flow-recombine/optical_flows/volume_0_8/flow_0000.png
Checking frame path: /kaggle/input/optical-flow-recombine/optical_flows/volume_0_9/flow_0000.png
Saved combined frame: /kaggle/

In [5]:
import shutil

# Specify the path to your folder
folder_path = '/kaggle/working/reassembled_video_volumes'
zip_file_path = '/kaggle/working/reassembled_video_volumes.zip'

# Create a zip archive of the folder
shutil.make_archive(zip_file_path.replace('.zip', ''), 'zip', folder_path)

'/kaggle/working/reassembled_video_volumes.zip'